In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 인코터 LSTM과 디코더 LSTM(Sequence to Sequence)로 번역기 만들기
- Google Neural Machine Translation(GNMT)
- RNN 기반의 Seq2Seq방식
- 자연어 생성 알고리즘

## 1. 패키지 import & 하이퍼파라미터
- 하이퍼파라미터 : 모델의 정확도 및 학습속도에 영향을 미치는 변수

In [7]:
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼파라미터
MY_HIDDEN = 128
MY_EPOCH = 500

## 2. 번역데이터 불러오기

In [8]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist() # 데이터프레임을 list로 변환
print('영어-한글 번역 데이터 :', eng_kor[:3])
print('영어-한글 번역 데이터 수 :', len(eng_kor))

영어-한글 번역 데이터 : [['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영어-한글 번역 데이터 수 : 110


## 3. 영어 알파벳과 한글문자 리스트 만들기

In [24]:
# 영어 알파벳 리스트(e_alpha)
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# print(e_alpha)
# { c:i for i, c in enumerate(e_alpha)}

# 한글 문자 리스트(k_ch)
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
k_ch.sort()
# print(k_ch)

k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()
k_alpha == k_ch # 순서와 내용이 모두 같음

# 순서는 무시하고 내용(요소들)만 같은지
from collections import Counter
list1 = ['가', '간', '나']
list2 = ['간', '나', '가']
Counter(list1) == Counter(list2)

True

In [28]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 :', alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) :', alpha_total_size)
print('한글 알파벳 갯수 :', len(k_alpha))

영어와 한글 알파벳 : ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩할 size) : 171
한글 알파벳 갯수 : 142


## 4. 문자당 num를 갖는 dict 만들기

In [34]:
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [41]:
data = eng_kor[0]
print(data)
print(char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d'])
print('인코더 입력(원핫인코딩전) :',[char_to_num[c] for c in data[0]])
print('디코더 입력(원핫인코딩전) :',[char_to_num[c] for c in 'S' + data[1]])
print('디코더 출력(원핫인코딩X) :', [char_to_num[c] for c in data[1]+'E'])

['cold', '감기']
5 17 14 6
인코더 입력(원핫인코딩전) : [5, 17, 14, 6]
디코더 입력(원핫인코딩전) : [0, 32, 46]
디코더 출력(원핫인코딩X) : [32, 46, 1]


In [44]:
# 희소행렬의 원핫인코딩방법 1 (희소행렬에서는 pd.get_dummies([2,9,7]) 안씀)
to_categorical([5,7,6,8],
              num_classes=10) # alpha_total_size

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [49]:
# 희소행렬의 원핫인코딩방법2
np.eye(10)[[5,7,6,8]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

## 5. 인코더 입력, 디코더 입력, 디코더 출력
- 인코더 입력데이터 : 영어 알파벳 -> 숫자 -> 원핫인코딩(110, 4, 171)
- 디코더 입력데이터 : 'S'+한글문자 -> 숫자 -> 원핫인코딩(110, 3, 171)
- 디코더 타겟데이터 : 한글문자+'E' -> 숫자 shape이 (110,3)인 list

In [60]:
def encoding(eng_kor=eng_kor):
    enc_in = []
    dec_in = []
    dec_out = []
    for data in eng_kor:
        # 인코더 입력데이터(영어 알파벳 -> 숫자 -> 원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]
        # print('영어 :', eng, eng_one)
        enc_in.append(eng_one) # eng_one의 shape : (4, 171)
        
        # 디코더 입력데이터("S한글" -> 숫자 -> 원핫인코딩)
        kor = [char_to_num[c] for c in "S"+data[1]]
        # kor_one = to_categorical(kor, num_classes=alpha_total_size)
        kor_one = np.eye(alpha_total_size)[kor] # kor_one의 shape : (3, 171)
        # print('한글 :', kor, kor_one)
        dec_in.append(kor_one)
        
        # 디코더 출력데이터('한글E' -> 숫자)
        kor = [char_to_num[c] for c in data[1]+"E"]
        # print(kor)
        dec_out.append(kor)
    return enc_in, dec_in, dec_out

In [69]:
sample = [['cold', '감시'], ['wood','나무']]
x_enc, x_dec, y_dec = encoding(sample)
X_enc = np.array(x_enc)
X_dec = np.array(x_dec)
Y_dec = np.array(y_dec)
X_enc.shape, X_dec.shape, Y_dec.shape # (2,3,1)

((2, 4, 171), (2, 3, 171), (2, 3))

In [70]:
# 축증가 방법1
Y_dec.reshape(2, 3, 1)

array([[[ 32],
        [107],
        [  1]],

       [[ 48],
        [ 83],
        [  1]]])

In [75]:
# 축증가 방법 2 : 증가하는 함수(맨 마지막 축)
np.expand_dims(Y_dec, axis=-1)

array([[[ 32],
        [107],
        [  1]],

       [[ 48],
        [ 83],
        [  1]]])

In [76]:
# 축 증가 방법 3
Y_dec[..., np.newaxis]

array([[[ 32],
        [107],
        [  1]],

       [[ 48],
        [ 83],
        [  1]]])

In [79]:
# 축증가 방법4
Y_dec[:,:, None]

array([[[ 32],
        [107],
        [  1]],

       [[ 48],
        [ 83],
        [  1]]])

## 6. 전체 입력데이터, 타겟데이터 준비

In [81]:
x_enc, x_dec, y_dec = encoding(eng_kor)
X_enc = np.array(x_enc)
X_dec = np.array(x_dec)
# Y_dec = np.array(y_dec).reshape(-1, 3, 1) # 차원추가
Y_dec = np.expand_dims(y_dec, axis=-1)
# Y_dec = np.array(y_dec)[..., np.newaxis]
# Y_dec = np.array(y_dec)[:,:,None]

X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

## 7. 모델구현

In [95]:
# 인코더 LSTM
ENC_IN = Input(shape=(4, alpha_total_size)) # alpha_total_size:171

_, state_h, state_c = LSTM(units=MY_HIDDEN,  # MY_HIDDEN:128
                           return_state=True, # return_state=True h값과 c 받기
                           # return_sequences=False # LSTM윗 출력 안 받음
                          )(ENC_IN)
# 인코더와 디코더 연결 고리
LINK = [state_h, state_c]

# 디코더 LSTM
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, # 128
              # return_state=False,
              return_sequences=True, # 윗 출력 받음
              )(DEC_IN,
               initial_state=LINK)

# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

# 모델
model = Model(inputs=[ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 4, 171)]     0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_14 (LSTM)                 [(None, 128),        153600      ['input_15[0][0]']               
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# 8. 학습과정 설정 & 학습

In [96]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'], # loss만 로그 출력
             ) 
begin = time()
model.fit([X_enc, X_dec], Y_dec,
         epochs=MY_EPOCH,
         verbose=1)
end = time()
print('학습시간 :', end-begin)

Epoch 1/500
4/4 [==============================] - 3s 11ms/step - loss: 5.1197 - accuracy: 0.2061
Epoch 2/500
4/4 [==============================] - 0s 10ms/step - loss: 4.9473 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 11ms/step - loss: 4.0616 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 11ms/step - loss: 3.4927 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4363 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3957 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 9ms/step - loss: 3.3623 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3336 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 11ms/step - loss: 3.3110 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 10ms/step - loss: 3.2987 - accuracy: 0.3333
Epoch 11/500
4/4 [==

4/4 [==============================] - 0s 9ms/step - loss: 1.1741 - accuracy: 0.8667
Epoch 85/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1545 - accuracy: 0.8333
Epoch 86/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1155 - accuracy: 0.8667
Epoch 87/500
4/4 [==============================] - 0s 9ms/step - loss: 1.0679 - accuracy: 0.8667
Epoch 88/500
4/4 [==============================] - 0s 9ms/step - loss: 1.0695 - accuracy: 0.8758
Epoch 89/500
4/4 [==============================] - 0s 9ms/step - loss: 1.0095 - accuracy: 0.8818
Epoch 90/500
4/4 [==============================] - 0s 9ms/step - loss: 0.9874 - accuracy: 0.8848
Epoch 91/500
4/4 [==============================] - 0s 13ms/step - loss: 0.9497 - accuracy: 0.9000
Epoch 92/500
4/4 [==============================] - 0s 10ms/step - loss: 0.9254 - accuracy: 0.8848
Epoch 93/500
4/4 [==============================] - 0s 10ms/step - loss: 0.8919 - accuracy: 0.9152
Epoch 94/500
4/4 [============

4/4 [==============================] - 0s 10ms/step - loss: 0.0407 - accuracy: 1.0000
Epoch 167/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0398 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0380 - accuracy: 0.9970
Epoch 169/500
4/4 [==============================] - 0s 8ms/step - loss: 0.0353 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0367 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0339 - accuracy: 1.0000
Epoch 173/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0359 - accuracy: 0.9970
Epoch 174/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 175/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0265 - accuracy: 1.0000
Epoch 176/500
4/

4/4 [==============================] - 0s 9ms/step - loss: 4.9148e-04 - accuracy: 1.0000
Epoch 248/500
4/4 [==============================] - 0s 11ms/step - loss: 4.5698e-04 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 10ms/step - loss: 6.2291e-04 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 14ms/step - loss: 6.0303e-04 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 8ms/step - loss: 6.9012e-04 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 8ms/step - loss: 7.1590e-04 - accuracy: 1.0000
Epoch 253/500
4/4 [==============================] - 0s 8ms/step - loss: 3.5134e-04 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 8ms/step - loss: 4.2882e-04 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 8ms/step - loss: 3.3203e-04 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 8ms/step - loss: 2.6551e-04 

4/4 [==============================] - 0s 8ms/step - loss: 8.1127e-06 - accuracy: 1.0000
Epoch 327/500
4/4 [==============================] - 0s 9ms/step - loss: 7.8331e-06 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 9ms/step - loss: 7.5462e-06 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 8ms/step - loss: 7.3616e-06 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 9ms/step - loss: 6.9199e-06 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 8ms/step - loss: 7.3974e-06 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 9ms/step - loss: 6.4597e-06 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 9ms/step - loss: 6.1551e-06 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 8ms/step - loss: 5.9420e-06 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 9ms/step - loss: 5.7086e-06 - a

4/4 [==============================] - 0s 9ms/step - loss: 1.2799e-06 - accuracy: 1.0000
Epoch 406/500
4/4 [==============================] - 0s 9ms/step - loss: 1.2726e-06 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 9ms/step - loss: 1.2492e-06 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 9ms/step - loss: 1.2362e-06 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 9ms/step - loss: 1.2257e-06 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 8ms/step - loss: 1.2134e-06 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1932e-06 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1823e-06 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 9ms/step - loss: 1.1704e-06 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 8ms/step - loss: 1.1571e-06 - a

4/4 [==============================] - 0s 9ms/step - loss: 6.3650e-07 - accuracy: 1.0000
Epoch 485/500
4/4 [==============================] - 0s 9ms/step - loss: 6.3253e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 8ms/step - loss: 6.2495e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 9ms/step - loss: 6.2242e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 8ms/step - loss: 6.1844e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 8ms/step - loss: 6.1302e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 8ms/step - loss: 6.0869e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 8ms/step - loss: 6.0869e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 8ms/step - loss: 6.0399e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 9ms/step - loss: 5.9749e-07 - a

In [97]:
model.evaluate([X_enc, X_dec], Y_dec)

4/4 [==============================] - 1s 6ms/step - loss: 5.6931e-07 - accuracy: 1.0000


[5.693144089491398e-07, 1.0]

## 9. 모델사용

In [101]:
# 쉬운문제
easy_test=[['cold','PP'],
           ['fact','PP'],
           ['love','PP'],
           ['luck','PP'],
           ['milk','PP']]
enc_in, dec_in, dec_out = encoding(easy_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((5, 4, 171), (5, 3, 171))

In [103]:
# 위의 문제 예측하기
pred = model.predict([enc_in, dec_in])
pred.argmax(axis=-1)

1/1 [==============================] - 0s 23ms/step


array([[ 32,  46,   1],
       [ 96, 109,   1],
       [ 96,  66,   1],
       [165, 125,   1],
       [124, 128,   1]], dtype=int64)

In [104]:
char_to_num['감'], alpha[32]

(32, '감')

In [106]:
# cold => 감기([32,46])

In [111]:
for test, yhat in zip(easy_test, pred):
#    print(test[0], yhat.argmax(axis=-1))
    eng = test[0]
    hat = np.argmax(yhat, axis=-1)
    kor = ''.join([alpha[h] for h in hat[:-1]])
    print("{}=>{}".format(eng, kor))

cold=>감기
fact=>사실
love=>사랑
luck=>행운
milk=>우유


In [115]:
# 어려운문제
hard_test=[['lvoe','PP'],
           ['loev','PP'],
           ['love','PP'],
           ['olve','PP'],
           ['eovl','PP']]
enc_in, dec_in, _ = encoding(hard_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
pred = model.predict([enc_in, dec_in]).argmax(axis=-1)
[''.join([alpha[h] for h in hat[:-1]]) for hat in pred]

1/1 [==============================] - 0s 24ms/step


['사랑', '사랑', '사랑', '사랑', '사다']

In [116]:
for test, yhat in zip(hard_test, pred):
    eng = test[0]
    kor = ''.join([alpha[h] for h in yhat[:-1]])
    print("{}=>{}({})".format(eng, kor, yhat[:-1]))

lvoe=>사랑([96 66])
loev=>사랑([96 66])
love=>사랑([96 66])
olve=>사랑([96 66])
eovl=>사다([96 60])
